imprements by https://www.kaggle.com/code/awsaf49/birdclef23-effnet-fsr-cutmixup-train

# Impoert library

In [1]:
import os
import random
from glob import glob
from pathlib import Path
import yaml
from tqdm import tqdm
import IPython.display as ipd

import numpy as np
import pandas as pd

import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt

import librosa
import wandb
from sklearn import metrics

import torch
from torch import nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler

from timm.scheduler import CosineLRScheduler

In [2]:
from config import CFG
from pytorch_model import BirdCLEF23Net
from pytorch_wav2logmel import Wav2Logmel
import pytorch_modeler as modeler
import pytorch_preprocessing as prep
import common as com

import model_soup

In [3]:
modeler.set_seed(CFG.seed)
# setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print('Debug :', CFG.debug)

cuda:0
Debug : False


In [4]:
# set path
base_path = '/kaggle/working/multi_label/exp23_secondary_label_2022_1st_like'
checkpoint_paths = [f"{base_path}/fold-{i}.pth" for i in range(5)]
# init model
model = BirdCLEF23Net(num_classes=CFG.num_classes)
# modelsoup (uniform)
uniform_model = model_soup.torch.uniform_soup(model, checkpoint_paths, device = 'cpu')

In [5]:
model_name = 'exp23_effnet_b1'
torch.save(uniform_model.state_dict(), f'{model_name}_modelsoups.pth')